# Scenario 03B — Incremental Help Notebook (AWS IAM & CloudTrail)

This notebook provides **optional, step‑by‑step hints** for Scenario 03B.

Each section includes:
- A conceptual hint
- A more concrete hint
- An optional reveal cell

Use this notebook only when you need it — the main scenario notebook is designed to be completed independently.

## 1. Understanding the AWS logs

**Goal:** Understand what `cloud_iam.csv` and `cloud_api.csv` represent.

### First Hint
Think of `cloud_iam.csv` as identity/configuration changes and `cloud_api.csv` as activity/usage.

### Second Hint
Inspect columns like `user`, `event_name`, `resource`, `region`, and `timestamp`.

### Reveal (optional)

In [ ]:
cloud_iam_df.head(), cloud_iam_df.info(), cloud_api_df.head(), cloud_api_df.info()

## 2. Timestamp normalization

**Goal:** Ensure timestamps are parsed correctly and usable for correlation.

### First Hint
Use `pd.to_datetime()` with `utc=True` to parse ISO‑8601 timestamps.

### Second Hint
Extract `hour` or `date` to help identify unusual activity times.

### Reveal (optional)

In [ ]:
cloud_iam_df['timestamp'] = pd.to_datetime(
    cloud_iam_df['timestamp'].astype(str).str.replace('Z', '', regex=False),
    utc=True,
    errors='coerce'
)
cloud_iam_df['hour'] = cloud_iam_df['timestamp'].dt.hour

cloud_api_df['timestamp'] = pd.to_datetime(
    cloud_api_df['timestamp'].astype(str).str.replace('Z', '', regex=False),
    utc=True,
    errors='coerce'
)
cloud_api_df['hour'] = cloud_api_df['timestamp'].dt.hour

## 3. Investigating IAM events

**Goal:** Identify suspicious IAM changes.

### First Hint
Look for events that modify privileges or authentication.

### Second Hint
Filter for events like `CreateAccessKey`, `AttachRolePolicy`, `AssumeRole`, or MFA‑related changes.

### Reveal (optional)

In [ ]:
susp_iam = cloud_iam_df[cloud_iam_df['event_name'].isin([
    'CreateAccessKey', 'AttachRolePolicy', 'AssumeRole', 'DeactivateMFADevice'
])]
susp_iam.head()

## 4. Investigating CloudTrail API calls

**Goal:** Identify suspicious API usage patterns.

### First Hint
Look for enumeration (`List*`, `Describe*`) and data access (`GetObject`).

### Second Hint
Check for unusual regions or activity shortly after IAM privilege changes.

### Reveal (optional)

In [ ]:
susp_api = cloud_api_df[cloud_api_df['event_name'].str.startswith('List') |
                        cloud_api_df['event_name'].str.startswith('Describe') |
                        cloud_api_df['event_name'].isin(['GetObject'])]
susp_api.head()

## 5. Correlating IAM and API events

**Goal:** Connect configuration changes to subsequent activity.

### First Hint
Did a user gain new privileges and then immediately perform unusual API calls?

### Second Hint
Filter API logs by the same user(s) involved in suspicious IAM events.

### Reveal (optional)

In [ ]:
actors = susp_iam['user'].unique().tolist()
related_api = cloud_api_df[cloud_api_df['user'].isin(actors)]
related_api.head()

## 6. Extracting IOCs

**Goal:** Identify concrete artifacts of malicious activity.

### First Hint
Look for suspicious users, roles, ARNs, IPs, or buckets.

### Second Hint
Focus on values that appear in both IAM and API suspicious events.

### Reveal (optional)

In [ ]:
susp_users = susp_iam['user'].unique().tolist()
susp_users

## 7. MITRE ATT&CK mapping

**Goal:** Describe the behavior using ATT&CK techniques.

### First Hint
Think about valid account use, privilege escalation, discovery, and data access.

### Second Hint
Common cloud‑relevant techniques include:
- `T1078` (Valid Accounts)
- `T1098` (Account Manipulation)
- `T1087` (Account Discovery)
- `T1530` (Data from Cloud Storage Object)

### Reveal (optional)

In [ ]:
example_mitre = ["T1078", "T1098", "T1530"]
example_mitre

## 8. Writing the triage summary

**Goal:** Tell the story of the incident.

### First Hint
Summarize what happened, how you know, and why it matters.

### Second Hint
Include:
- Initial access (if visible)
- Privilege escalation or misuse
- Data access or exfiltration
- Key IOCs and MITRE techniques

### Reveal (optional)

In [ ]:
example_summary = """
An IAM user appears to have been compromised and used to modify account
permissions, create new access keys, and assume a high‑privilege role.
Shortly after, the actor performed broad enumeration of IAM and S3 resources
and accessed sensitive objects, suggesting data exfiltration risk.
"""
example_summary